In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as f
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from sklearn import preprocessing
from tqdm import tqdm
from sklearn.preprocessing import FunctionTransformer
import seaborn as sns 
from scipy import stats
import statsmodels.api as sm

from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor, BaggingRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import ElasticNet, Lasso
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import SVR
from sklearn.model_selection import KFold, cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import RobustScaler
import h2o
from h2o.automl import H2OAutoML
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.neural_network import MLPRegressor
from scipy.stats import norm
import copy
from sklearn.model_selection import KFold
from catboost import CatBoostRegressor, Pool, metrics, cv
import xgboost as xgb
from scipy.stats import gmean
from sklearn.impute import KNNImputer
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

import datetime
from math import floor
import tensorflow as tf
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"
print("tf version:",tf.__version__)
print(tf.test.gpu_device_name())
from tensorflow.keras.layers import Input, TimeDistributed, Dense, Conv2D, LSTM, Flatten, Reshape, Embedding
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import Huber
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

import xgboost
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from lightgbm import plot_importance



ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
import tensorflow as tf

In [ ]:
global_region_tensor = np.load('./train_global_region_tensor_scaled_sg5.npy', allow_pickle=True)
column_df = pd.read_csv('./column_names.csv')
region_df = pd.read_csv('./target_points.csv')
col_names = column_df['name']
target_regions = region_df['region_id']

In [ ]:
temporal_attrs = ['year', 'month', 'dayofyear', 'season', 'day_of_year_sin', 'day_of_year_cos','month_sin', 'month_cos', 'season_sin', 'season_cos']

In [ ]:
print("Global region tensor shape:", global_region_tensor.shape)

In [ ]:
target_regions

In [ ]:
region_df['lon_index'].max()

In [ ]:
MAX_SG = 5
NUM_TIMESTEPS = global_region_tensor.shape[0]
NUM_REGIONS = len(target_regions)
NUM_FEATURES = global_region_tensor.shape[3]
TEMPORAL_FEATURES = len(temporal_attrs)
SPATIAL_FEATURES = NUM_FEATURES-TEMPORAL_FEATURES - 3 - 1
PADDED_LAT = global_region_tensor.shape[1]
PADDED_LON = global_region_tensor.shape[2]
TRAIN_SPLIT = NUM_TIMESTEPS - 2*30
DROPOUT_RATE = 0.3
VALID_SPLIT = NUM_TIMESTEPS - 30
SG = 5
SPATIAL_WIDTH = 2*SG+1
target_region_ids = []

for region in target_regions:
  tmp_list = region.split(', ')
  tmp_list = [_.strip() for _ in tmp_list]
  region_0 = int(tmp_list[0][1:])
  region_1 = int(tmp_list[1][:-1])
  a = region_df['lat_index'].max()-region_0+MAX_SG
  b = region_1-region_df['lon_index'].min()+MAX_SG
  target_region_ids.append((a, b))

BATCH_SIZE = 512
BUFFER_SIZE = 15000
PREFETCH_SIZE = 5
EPOCHS = 100

In [ ]:
CONV2D_LAYERS = 1
CONV1D_LAYERS = 0
RGN_ID_EMB_DIM = 8
SPTL_CONV_FILTERS = 16
SPTL_KERNEL_SIZE = 2
SPTL_STRIDE = 1
SPTL_PADDING = 'same'
SPTL_OUTPUT_DIM = 24
CONV1D_FILTERS = 16
REC_DROPOUT_RATE = 0
FC1_UNITS = 128
FC2_UNITS = 128
PRED_TAR = 'tmp2m'
TAR_VAR = 'tmp2m'
MODEL_NAME = PRED_TAR+'_sp-tm_sg'+str(SG)+'_emb'+str(RGN_ID_EMB_DIM)
TIMESTAMP = '{date:%Y_%m_%d_%H_%M}'.format(date = datetime.datetime.now())

In [ ]:
rgn_id_vocab = [str(region) for region in target_regions]
rgn_id_to_int = {rgn_id:i for i, rgn_id in enumerate(rgn_id_vocab)}
int_to_rgn_id = {i:rgn_id for i, rgn_id in enumerate(rgn_id_vocab)}

In [ ]:
print("Prediction target: ",PRED_TAR,
      "\nEpochs: ",EPOCHS,
      "\nBatch size: ",BATCH_SIZE,
      "\nSpatial granularity: ", SG,
      "\nMax spatial granularity: ", MAX_SG,
      "\nModel name: ", MODEL_NAME)

In [ ]:
train_df = pd.read_csv('./n_train.csv')
test_df = pd.read_csv('./n_test.csv')
train_all_date_list = train_df['startdate'].unique()
test_all_date_list = test_df['startdate'].unique()

In [ ]:
locations_df = pd.read_csv('./locations.csv')

In [ ]:
target = 'contest-tmp2m-14d__tmp2m'

In [ ]:
lat_max = locations_df['lat_index'].max()
lon_min = locations_df['lon_index'].min()
print(lat_max, lon_min)

In [ ]:
region_df

In [ ]:
location = locations_df[(locations_df['lat_index'] == 47) & (locations_df['lon_index'] == 268)]
location

In [ ]:
train_df[(train_df['loc_group'] == 456) & (train_df['startdate'] == '2016-01-19')]

In [ ]:
def get_target_data_by_date_index_and_region(target_region, date):
  # print(target_region, lat_max + MAX_SG - target_region[0], target_region[1] + lon_min - MAX_SG, date)
  location = locations_df[(locations_df['lat_index'] == lat_max + MAX_SG - target_region[0]) & (locations_df['lon_index'] == target_region[1] + lon_min - MAX_SG)]
  if len(location) == 0:
    return 0
  loc_group = location['loc_group'].tolist()[0]
  # print(target_region, lat_max + MAX_SG - target_region[0], target_region[1] + lon_min - MAX_SG, date, loc_group)
  sample = train_df[(train_df['loc_group'] == loc_group) & (train_df['startdate'] == date)]
  # print(sample)
  return sample[target].tolist()[0]

In [ ]:
t = [0 for i in range(68)]

In [ ]:
len(t[3:-TEMPORAL_FEATURES - 1])
len(t[-TEMPORAL_FEATURES - 1:-1])

In [ ]:
def window_single_spatial_temporal_target(dataset, target_region, index, sg, date_list, seq_len, tar_var=TAR_VAR,
                                           region_emb=True):

    local_region = dataset[:, (target_region[0] - sg):(target_region[0] + sg + 1),
                   (target_region[1] - sg):(target_region[1] + sg + 1), :]
    

    spatial_data = local_region[[index], :, :, 3:-TEMPORAL_FEATURES - 1].astype(np.float16)

    if region_emb:
        local_reg_id = str((int(local_region[0, sg, sg, 1]), int(local_region[0, sg, sg, 2])))
        local_reg_id = rgn_id_to_int[local_reg_id]
        temporal_data = local_region[[index], sg, sg, -TEMPORAL_FEATURES - 1:-1].astype(np.float16)
        region_embedding = np.repeat(local_reg_id, 1).reshape(1, 1).astype(np.int16)
    target_data = get_target_data_by_date_index_and_region(target_region, date_list[index])
    # print(target_region, date_list[index], target_data)

    return np.array(spatial_data), np.array(temporal_data), np.array(region_embedding), np.array(target_data)

In [ ]:
def generate_input_batch(dataset, batch_size, sg,  tar_var, date_list, seq_len):

    spatial_data, temporal_data, reg_emb_data, target_data = [], [], [], []
    for batch in range(batch_size):
        rand_region_idx = np.random.choice(NUM_REGIONS, 1, replace=False)[0]
        rand_region = target_region_ids[rand_region_idx]
        rand_date_idx = np.random.choice(NUM_TIMESTEPS, 1, replace=False)[0]
        spt, tmp, remb, tar = window_single_spatial_temporal_target(dataset, rand_region, rand_date_idx, sg, date_list,
                                                                     seq_len, tar_var)
        spatial_data.append(spt)
        temporal_data.append(tmp)
        reg_emb_data.append(remb)
        target_data.append(tar)

    spatial_data = np.stack(spatial_data)
    temporal_data = np.stack(temporal_data)
    reg_emb_data = np.stack(reg_emb_data)
    target_data = np.stack(target_data)

    # yield  ({"spatial_input": tf.convert_to_tensor(spatial_data), "temporal_input": tf.convert_to_tensor(temporal_data), "region_id_input": tf.convert_to_tensor(reg_emb_data)},
    #        {"target_output": tf.convert_to_tensor(target_data)})

    yield ({"spatial_input": spatial_data, "temporal_input": temporal_data, "region_id_input": reg_emb_data},
           {"target_output": target_data})

In [ ]:
res = generate_input_batch(dataset=global_region_tensor, batch_size=BATCH_SIZE, sg=SG, tar_var=TAR_VAR, date_list= train_all_date_list, seq_len=1)


In [ ]:
SEQ_LEN = 1
dataset_generator = tf.data.Dataset.from_generator(
    generator=lambda: generate_input_batch(dataset=global_region_tensor, batch_size=BATCH_SIZE, sg=SG, tar_var=TAR_VAR, date_list= train_all_date_list, seq_len=SEQ_LEN),
    output_types=({"spatial_input":np.float64,"temporal_input":np.float64,"region_id_input":np.int16},{"target_output":np.float64}),
    output_shapes=({"spatial_input":[BATCH_SIZE,SEQ_LEN, SPATIAL_WIDTH,SPATIAL_WIDTH,SPATIAL_FEATURES], \
                    "temporal_input":[BATCH_SIZE,SEQ_LEN, TEMPORAL_FEATURES],
                    "region_id_input":[BATCH_SIZE,SEQ_LEN, 1]}, \
                    {"target_output":[BATCH_SIZE,]}))

In [ ]:
dataset_generator = dataset_generator.prefetch(PREFETCH_SIZE).repeat()

In [ ]:
def build_sptl_tmpl_model(sptl_width=SPATIAL_WIDTH, sptl_features=SPATIAL_FEATURES,
                          tmpl_features=TEMPORAL_FEATURES, batch_size=BATCH_SIZE,
                          sptl_conv_filters=SPTL_CONV_FILTERS, sptl_kernel_size=SPTL_KERNEL_SIZE,
                          sptl_stride=SPTL_STRIDE, sptl_padding=SPTL_PADDING, sptl_output_dim=SPTL_OUTPUT_DIM,
                          rgn_id_vocab=NUM_REGIONS, rgn_id_emb_dim=RGN_ID_EMB_DIM, dropout_rate=DROPOUT_RATE, model_name=MODEL_NAME,
                          print_summary=True):

    spatial_input = Input(shape=[1, sptl_width, sptl_width, sptl_features], batch_size=batch_size,
                          name='spatial_input')

    spatial_model = Conv2D(filters=sptl_conv_filters, kernel_size=(sptl_kernel_size, sptl_kernel_size),
                                           strides=(sptl_stride, sptl_stride), padding=sptl_padding,
                                           name='sptl_conv2d')(spatial_input)

    spatial_model = Flatten(name='sptl_flatten')(spatial_model)

    spatial_model = Reshape(target_shape=(1, -1), name='sptl_reshape')(spatial_model)

    spatial_output = Dense(units=sptl_output_dim, activation='relu', name='sptl_emb_out')(spatial_model)

    # Temporal Model Input
    temporal_input = Input(shape=(1, tmpl_features), batch_size=batch_size, name='temporal_input')

    # Region ID Input
    region_id_input = Input(shape=(1, 1), batch_size=batch_size, name='region_id_input')

    region_id_emb = Reshape((1,), input_shape=(1, 1), name='region_id_reshape')(region_id_input)

    region_id_emb = Embedding(input_dim=rgn_id_vocab, input_length=1, mask_zero=False, batch_size=batch_size,
                              output_dim=rgn_id_emb_dim, name='durations_emb_layer')(region_id_emb)

    # Concat Inputs
    concat = K.concatenate([spatial_output, temporal_input, region_id_emb], axis=-1)

    # LSTM Architecture
    # lstm_model = LSTM(units=lstm1_units, return_sequences=True, name='lstm_1')(concat)

    # lstm_model = LSTM(units=lstm2_units, return_sequences=False, name='lstm_2')(lstm_model)

    model_output = Dense(units=1, name='target_output')(concat)

    model = Model(inputs=[spatial_input, temporal_input, region_id_input], outputs=[model_output], name=model_name)

    if print_summary:
        print(model.summary())

    return (model)

spatial_temporal_model = build_sptl_tmpl_model()

In [ ]:
STEPS_PER_EPOCH = int(len(train_df) / BATCH_SIZE)
STEPS_PER_EPOCH

In [ ]:
RUN_ID = "1"
def train_spatial_temporal_model(model, dataset_generator, opt='adam', epochs=EPOCHS, steps_per_epoch=STEPS_PER_EPOCH,
                                 include_tb=False): 

    ## Early stopping
    earlystopping = EarlyStopping(monitor='loss', min_delta=0.00001, patience=10, restore_best_weights=True)  # val_loss

    # Automatically save latest best model to file
    filepath =  "./models/model_saves/" + PRED_TAR + '/' + RUN_ID + ".hdf5"
    checkpoint = ModelCheckpoint(filepath=filepath, monitor='loss', verbose=0, save_best_only=True, mode='min')

    # Set callbacks
    callbacks_list = [checkpoint, earlystopping]

    # Optimizers
    optimizers = {'adam': Adam(learning_rate=0.03, beta_1=0.9, beta_2=0.999, amsgrad=False)}

    model.compile(loss='mean_absolute_error', optimizer=optimizers[opt],
                  metrics=[RootMeanSquaredError(), Huber()])

    # Fit model #x = [spatial_train, temporal_train_x], y = temporal_train_y,
    history = model.fit(dataset_generator, epochs=epochs, use_multiprocessing=True,
                        steps_per_epoch=steps_per_epoch, verbose=1, callbacks=callbacks_list)
    return (history)

## Train ##
history = train_spatial_temporal_model(model = spatial_temporal_model, dataset_generator = dataset_generator,
                                       opt = 'adam', steps_per_epoch=500, include_tb=False)
